In [1]:
%config IPCompleter.use_jedi=False

In [2]:
from pc_zap_scrapper.v2.scrape import get_estates_from_page

In [3]:
import nest_asyncio
nest_asyncio.apply()

estates = await get_estates_from_page(
    action="venda",
    type="imoveis",
    localization="mg+pocos-de-caldas",
    page=10,
)

estates

Loop 7 | 105 elementos: 100%|██████████| 105/105 [01:07<00:00,  1.56it/s]


,id,action,search_date,post_type,link,type,image_list,snippet,street,neighbor,city,state,floor_size,number_of_rooms,number_of_bathrooms,number_of_parking_spaces,amenities_list,price,condominium,iptu
0,2715278498,venda,2024-11-26 04:12:01.635399,SUPER PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 4 quartos à venda,Rua Doutor Francisco Faria Lobato,Centro,Poços de Caldas,MG,258,4.0,5.0,2.0,"[Aceita animais, Closet, Mobiliado, Academia, ...",1760000.0,1.0,3.0
1,2701686173,venda,2024-11-26 04:12:01.635399,SUPER PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 3 quartos à venda,,São Benedito,Poços de Caldas,MG,160,3.0,4.0,2.0,None,1750000.0,760.0,0.0
2,2696155120,venda,2024-11-26 04:12:01.635399,SUPER PREMIUM,https://www.zapimoveis.com.br/imovel/venda-ter...,Lote,[https://resizedimgs.zapimoveis.com.br/crop/61...,Lote/Terreno à venda,,Jardim Europa,Poços de Caldas,MG,502,NaN,NaN,NaN,None,790000.0,NaN,NaN
3,2751418766,venda,2024-11-26 04:12:01.635399,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-cas...,None,[https://resizedimgs.zapimoveis.com.br/crop/61...,Imóvel comercial à venda,,Centro,Poços de Caldas,MG,121,NaN,2.0,1.0,None,1300000.0,NaN,NaN
4,2705454927,venda,2024-11-26 04:12:01.635399,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 3 quartos à venda,,Centro,Poços de Caldas,MG,144,3.0,3.0,2.0,None,1875000.0,758.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2500689482,venda,2024-11-26 04:12:01.635399,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-gal...,None,[https://resizedimgs.zapimoveis.com.br/crop/61...,Galpão/Depósito/Armazém à venda,Avenida José Remígio Prezia,Jardim dos Estados,Poços de Caldas,MG,708,NaN,NaN,NaN,None,2300000.0,NaN,NaN
101,2680997706,venda,2024-11-26 04:12:01.635399,STANDARD,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 2 quartos à venda,,Jardim Quisisana,Poços de Caldas,MG,69,2.0,2.0,1.0,"[Varanda, Cozinha]",320000.0,NaN,NaN
102,2731796035,venda,2024-11-26 04:12:01.635399,STANDARD,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 2 quartos à venda,,Jardim Country Club,Poços de Caldas,MG,72,2.0,2.0,2.0,"[Churrasqueira, Elevador, Varanda gourmet, Int...",550000.0,NaN,NaN
103,2647900384,venda,2024-11-26 04:12:01.635399,STANDARD,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 2 quartos à venda,,Jardim das Hortênsias,Poços de Caldas,MG,86,2.0,1.0,1.0,"[Condomínio fechado, Cozinha]",310000.0,1.0,1.0


In [15]:
from datetime import datetime
from typing import get_args, Literal

import pandas as pd
from pc_zap_scrapper.v2.config import config, REAL_ESTATE_TYPES, ACTION_TYPES

async def get_estates_from_page(
    action: ACTION_TYPES,
    type: REAL_ESTATE_TYPES,
    localization: str,
    page: int,
):
    if action != "venda":
        raise NotImplementedError(
            f"Action {action} not implemented yet. Available actions are: {list(get_args(ACTION_TYPES))}"
        )

    divs = await scrape_estate_divs_from_page(
        action=action,
        type=type,
        localization=localization,
        page=page,
    )

    search_date = datetime.now()
    estate_info = [get_info_from_div(div, action, search_date).model_dump() for div in divs]
    return pd.DataFrame(estate_info)

In [16]:
import nest_asyncio
nest_asyncio.apply()

estates = await get_estates_from_page(
    action="venda",
    type="imoveis",
    localization="mg+pocos-de-caldas",
    page=1,
)

Loop 8 | 112 elementos: 100%|██████████| 112/112 [01:15<00:00,  1.49it/s]


In [3]:
import nest_asyncio
nest_asyncio.apply()

divs = await scrape_estate_divs_from_page(
    action="venda",
    type="imoveis",
    localization="mg+pocos-de-caldas",
    page=1,
)

Loop 8 | 113 elementos: 100%|██████████| 113/113 [01:13<00:00,  1.54it/s]


,id,action,search_date,post_type,link,type,image_list,snippet,street,neighbor,city,state,floor_size,number_of_rooms,number_of_bathrooms,number_of_parking_spaces,amenities_list,price,condominium,iptu
69,2696155526,venda,2024-11-26 03:54:46.485465,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-ter...,Lote,[https://resizedimgs.zapimoveis.com.br/crop/61...,Lote/Terreno à venda,,Jardim Europa,Poços de Caldas,MG,538,NaN,NaN,NaN,None,650000.0,NaN,NaN
2,2757656026,venda,2024-11-26 03:54:46.485465,SUPER PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 2 quartos à venda,,Jardim Country Club,Poços de Caldas,MG,62,2.0,2.0,1.0,None,380000.0,NaN,NaN
64,2757607816,venda,2024-11-26 03:54:46.485465,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 3 quartos à venda,,Jardim Elvira Dias,Poços de Caldas,MG,92,3.0,2.0,1.0,"[Cozinha, Varanda gourmet, Área de serviço]",550000.0,350.0,0.0
84,2673458698,venda,2024-11-26 03:54:46.485465,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-apa...,Apartamento,[https://resizedimgs.zapimoveis.com.br/crop/61...,Apartamento com 2 quartos à venda,,Santa Teresa,Poços de Caldas,MG,70,2.0,2.0,1.0,"[Cozinha, Área de serviço]",240000.0,NaN,NaN
41,2752641599,venda,2024-11-26 03:54:46.485465,PREMIUM,https://www.zapimoveis.com.br/imovel/venda-ter...,Lote,[https://resizedimgs.zapimoveis.com.br/crop/61...,Lote/Terreno à venda,Avenida Castorina Fernandes Vieira,Jardim Vitória,Poços de Caldas,MG,766,NaN,NaN,NaN,None,450000.0,NaN,NaN


# utils

In [1]:
import logging
import traceback
from datetime import datetime, timezone
    
def suppress_errors_and_log(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"An error occurred in {func.__name__}: {e}")
            logging.error(traceback.format_exc())
            return None
    return wrapper


def datetime_to_iso8601_z(dt):
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    else:
        dt = dt.astimezone(timezone.utc)
    return dt.strftime('%Y-%m-%dT%H:%M:%SZ')


# config

In [5]:
import os
from dotenv import load_dotenv
from pydantic_settings import BaseSettings
from pydantic import Field


class Config(BaseSettings):
    MAX_ESTATES_PER_PAGE: int = Field(default=200, env="MAX_ESTATES_PER_PAGE") 
    LOGURU_LEVEL: str = Field(default="INFO", env="LOGURU_LEVEL")


load_dotenv()
config = Config()

# scrape

In [ ]:
from tqdm.asyncio import tqdm_asyncio
from bs4 import BeautifulSoup


async def simulate_user_actions(page):
    for _ in range(random.randint(3, 7)):
        x, y = random.randint(0, 800), random.randint(0, 600)
        await page.mouse.move(x, y)
        await random_delay(0.1, 0.5)

async def scroll_position(page):
    await page.wait_for_selector('[data-position]')
    await page.evaluate("""
        () => {
            const elements = document.querySelectorAll('[data-position]');
            if (elements.length > 0) {
                elements[elements.length - 1].scrollIntoView();
            }
        }
    """)
    await simulate_user_actions(page)
    await random_delay(3, 5)

async def fetch_page_content(page):
    html_page = await page.content()
    return BeautifulSoup(html_page, "html.parser")

async def perform_user_actions(page):
    await simulate_user_actions(page)
    await random_delay(1, 3)

async def scroll_and_get_divs(page):
    await scroll_position(page)
    soup = await fetch_page_content(page)
    return soup.find_all("div", attrs={"data-position": True})

async def progress_loop(page, max_pages=200, max_iterations=20):
    n_pages = 0
    count = 1

    with tqdm_asyncio(total=max_iterations, desc="Progresso") as pbar:
        while n_pages < max_pages and count <= max_iterations:
            print(f"Loop {count}")
            await perform_user_actions(page)
            divs = await scroll_and_get_divs(page)
            print(f"{len(divs)} elementos\n\n")
            
            if n_pages == len(divs):  # Checa se o progresso parou
                print("Nenhum novo elemento encontrado, encerrando.")
                break

            n_pages = len(divs)
            count += 1
            pbar.update(1)

async def main():
    """Função principal para iniciar o loop."""
    # Exemplo de inicialização do navegador (depende da sua configuração)
    # browser = await launch_browser()
    # page = await browser.new_page()
    page = None  # Substitua com o objeto real da página
    await progress_loop(page)

# asyncio.run(main())


In [16]:
from pydantic import BaseModel

class ZapImoveisURL(BaseModel):
    base_url: str = "https://www.zapimoveis.com.br"
    action: str
    type: str
    localization: str
    page: str | int | None = None

    @property
    def text(self):
        _page = f"?pagina={self.page}" if self.page else ""
        return f"{self.base_url}/{self.action}/{self.type}/{self.localization}/{_page}"

    def __str__(self):
        return self.text

    def __repr__(self):
        return self.text

    def __eq__(self, other):
        if isinstance(other, str):
            return self.text == other
        return super().__eq__(other)

    def __getattr__(self, item):
        if hasattr(self.text, item):
            return getattr(self.text, item)
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{item}'")

    def __add__(self, other):
        if isinstance(other, str):
            return self.text + other
        return NotImplemented

    def __radd__(self, other):
        if isinstance(other, str):
            return other + self.text
        return NotImplemented

url = ZapImoveisURL(action="venda", type="imoveis", localization="mg+pocos-de-caldas", page=1)

url

https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1

'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1sasda'

In [17]:
url + "q"

'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1q'

In [ ]:
import asyncio
import random
from playwright.async_api import async_playwright

from random_user_agent.user_agent import UserAgent

import nest_asyncio
nest_asyncio.apply()

async def scrape_zap_imoveis_page(action, type, localization, page):
    url = f"https://www.zapimoveis.com.br/{action}/{type}/{localization}/?pagina={page}"
    
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=["--disable-blink-features=AutomationControlled"]
        )
        context = await browser.new_context(
            viewport={"width": 1366, "height": 768},
            user_agent=UserAgent().get_random_user_agent()
        )

        page = await context.new_page()

        await page.set_extra_http_headers({
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/"
        })

        await page.goto(url)


        async def simulate_user_actions():
            for _ in range(random.randint(3, 7)):
                x, y = random.randint(0, 800), random.randint(0, 600)
                await page.mouse.move(x, y)
                await random_delay(0.1, 0.5)

        async def scroll_position():
            await page.wait_for_selector('[data-position]')
            await page.evaluate("""
                () => {
                    const elements = document.querySelectorAll('[data-position]');
                    if (elements.length > 0) {
                        elements[elements.length - 1].scrollIntoView();
                    }
                }
            """)
            await simulate_user_actions()
            await random_delay(3, 5)

        n_pages = 0
        count = 1
        html_page = None
        while (n_pages < 200) and (count < 20):
            print(f"Loop {count}")
            await simulate_user_actions()
            await random_delay(1, 3)
            await scroll_position()
            html_page = await page.content()
            soup = BeautifulSoup(html_page, "html.parser")
            divs = soup.find_all("div", attrs={"data-position": True})
            print(f"{len(divs)} elementos\n\n")
            if n_pages == len(divs):
                break
            n_pages = len(divs)
            count += 1
        
        return html_page

# Função para introduzir atrasos aleatórios
async def random_delay(min_delay=1, max_delay=3):
    await asyncio.sleep(random.uniform(min_delay, max_delay))


In [ ]:

import nest_asyncio
nest_asyncio.apply()